In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
data_transform = {
    'train': transforms.Compose(transforms=[
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ]),
    'val': transforms.Compose(transforms=[
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])
}

Data source: https://www.kaggle.com/gauravduttakiit/ants-bees

In [8]:
data_dir = '../data/hymenoptera_data'
sets  = ['train', 'val']
imgs_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transform[x]) for x in sets}
data_loaders = {x: torch.utils.data.DataLoader(imgs_datasets[x], batch_size=4, shuffle=True, num_workers=1) for x in sets} 
dataset_sizes = {x: len(imgs_datasets[x]) for x in sets}
classes_names = imgs_datasets['train'].classes
print(classes_names)

['ants', 'bees']


In [11]:
def train_model(model, criterion, optimizer, scheduler, n_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}/{n_epochs}')
        print('-'*10)
        for phase in sets:
            if phase == 'train':
                model.train() # set model to train mode
            else:
                model.eval() # set model to evaluation mode

            running_loss = 0.0
            running_corrects = 0.0

            for imgs, labels in data_loaders[phase]:
                imgs = imgs.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(imgs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * imgs.size(0)
                running_corrects += torch.sum(preds == labels)
            if phase == 'train':
                scheduler.step() # update scheduler
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Load pre-train model and replace last layer

In [13]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features # number of features for last fully connected layer
model.fc = nn.Linear(in_features=num_features, out_features=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=model.parameters(), lr=0.001)

# scheduler to update learing rate
step_lr_scheduler = lr_scheduler.StepLR(optimizer=optimizer, step_size=7, gamma=0.1) # every 7 epoch, lr = lr * gamma

model = train_model(model=model, criterion=criterion, optimizer=optimizer, scheduler=step_lr_scheduler, n_epochs=20)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/wenhan/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


Epoch 1/20
----------
train Loss: 0.6530 Acc: 0.6025
val Loss: 0.4555 Acc: 0.8366

Epoch 2/20
----------
train Loss: 0.5257 Acc: 0.7541
val Loss: 0.3619 Acc: 0.8954

Epoch 3/20
----------
train Loss: 0.4684 Acc: 0.8115
val Loss: 0.2863 Acc: 0.9150

Epoch 4/20
----------
train Loss: 0.4049 Acc: 0.8033
val Loss: 0.2487 Acc: 0.9150

Epoch 5/20
----------
train Loss: 0.4229 Acc: 0.8320
val Loss: 0.2292 Acc: 0.9346

Epoch 6/20
----------
train Loss: 0.3735 Acc: 0.8525
val Loss: 0.2245 Acc: 0.9085

Epoch 7/20
----------
train Loss: 0.3706 Acc: 0.8197
val Loss: 0.2026 Acc: 0.9216

Epoch 8/20
----------
train Loss: 0.3118 Acc: 0.8770
val Loss: 0.1992 Acc: 0.9477

Epoch 9/20
----------
train Loss: 0.4111 Acc: 0.7992
val Loss: 0.2235 Acc: 0.9216

Epoch 10/20
----------
train Loss: 0.3233 Acc: 0.8607
val Loss: 0.2137 Acc: 0.9020

Epoch 11/20
----------
train Loss: 0.3127 Acc: 0.8770
val Loss: 0.1959 Acc: 0.9412

Epoch 12/20
----------
train Loss: 0.3565 Acc: 0.8197
val Loss: 0.1962 Acc: 0.9346

E

# Freeze all layers except the last layer (faster)

In [14]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad= False
num_features = model.fc.in_features # number of features for last fully connected layer
model.fc = nn.Linear(in_features=num_features, out_features=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=model.parameters(), lr=0.001)

# scheduler to update learing rate
step_lr_scheduler = lr_scheduler.StepLR(optimizer=optimizer, step_size=7, gamma=0.1) # every 7 epoch, lr = lr * gamma

model = train_model(model=model, criterion=criterion, optimizer=optimizer, scheduler=step_lr_scheduler, n_epochs=20)

Epoch 1/20
----------
train Loss: 0.6691 Acc: 0.5943
val Loss: 0.5757 Acc: 0.6863

Epoch 2/20
----------
train Loss: 0.5839 Acc: 0.6885
val Loss: 0.4769 Acc: 0.8039

Epoch 3/20
----------
train Loss: 0.5423 Acc: 0.7459
val Loss: 0.3928 Acc: 0.8954

Epoch 4/20
----------
train Loss: 0.5119 Acc: 0.7869
val Loss: 0.3550 Acc: 0.9020

Epoch 5/20
----------
train Loss: 0.4592 Acc: 0.7992
val Loss: 0.3238 Acc: 0.9216

Epoch 6/20
----------
train Loss: 0.4697 Acc: 0.7705
val Loss: 0.2942 Acc: 0.9477

Epoch 7/20
----------
train Loss: 0.4944 Acc: 0.7336
val Loss: 0.2842 Acc: 0.9542

Epoch 8/20
----------
train Loss: 0.4067 Acc: 0.8648
val Loss: 0.2699 Acc: 0.9412

Epoch 9/20
----------
train Loss: 0.4244 Acc: 0.8279
val Loss: 0.2750 Acc: 0.9673

Epoch 10/20
----------
train Loss: 0.3906 Acc: 0.8484
val Loss: 0.2716 Acc: 0.9608

Epoch 11/20
----------
train Loss: 0.3869 Acc: 0.8770
val Loss: 0.2696 Acc: 0.9412

Epoch 12/20
----------
train Loss: 0.4180 Acc: 0.8279
val Loss: 0.2729 Acc: 0.9542

E